In [1]:
import numpy as np

In [2]:
path: str = "../data.nosync/usilaughs/data_left.npz"
data = np.load(path, allow_pickle=True)
X = data["values"]

In [3]:
import timesfm
# For Torch
tfm = timesfm.TimesFm(
      hparams=timesfm.TimesFmHparams(
          backend="cpu",
          per_core_batch_size=32,
          horizon_len=128,
          num_layers=50,
        #   use_positional_embedding=False,
          context_len=2048,
      ),
      checkpoint=timesfm.TimesFmCheckpoint(
          huggingface_repo_id="google/timesfm-2.0-500m-pytorch"),
  )


TimesFM v1.2.0. See https://github.com/google-research/timesfm/blob/master/README.md for updated APIs.
Loaded Jax TimesFM.
Loaded PyTorch TimesFM.


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [4]:
out = tfm.forecast(inputs=X[[0],...,0])

In [5]:
print(out[0].shape) 
print(out[1].shape) 

torch.Size([16, 128, 10])
torch.Size([16, 128, 10])


In [8]:
print(out[1])
print(out[1].reshape(-1))

tensor([[[1.1427, 1.1581, 1.1513,  ..., 1.1329, 1.1277, 1.1057],
         [1.1413, 1.1439, 1.1403,  ..., 1.1417, 1.1336, 1.1096],
         [1.1343, 1.1406, 1.1399,  ..., 1.1315, 1.1128, 1.0869],
         ...,
         [1.1768, 1.1317, 1.1568,  ..., 1.1907, 1.1862, 1.1644],
         [1.1784, 1.1559, 1.1594,  ..., 1.1912, 1.1789, 1.1462],
         [1.1844, 1.1447, 1.1481,  ..., 1.1954, 1.1858, 1.1634]],

        [[1.1446, 1.1649, 1.1558,  ..., 1.1314, 1.1269, 1.1071],
         [1.1412, 1.1493, 1.1439,  ..., 1.1373, 1.1306, 1.1084],
         [1.1351, 1.1419, 1.1407,  ..., 1.1295, 1.1143, 1.0910],
         ...,
         [1.1814, 1.1431, 1.1621,  ..., 1.1965, 1.1934, 1.1718],
         [1.1793, 1.1599, 1.1601,  ..., 1.1946, 1.1849, 1.1546],
         [1.1819, 1.1522, 1.1514,  ..., 1.1940, 1.1853, 1.1629]],

        [[1.1526, 1.1733, 1.1634,  ..., 1.1366, 1.1334, 1.1171],
         [1.1465, 1.1570, 1.1507,  ..., 1.1394, 1.1335, 1.1152],
         [1.1380, 1.1458, 1.1432,  ..., 1.1292, 1.1159, 1.

# TimeMixer Granite IBM

In [3]:
from transformers import PatchTSMixerForPrediction
import numpy as np
import torch

In [4]:

# Example usage of PatchTSMixerModel
path: str = "../data.nosync/apsync/data_engagement_10sec.npz"
data = np.load(path, allow_pickle=True)
X = data["values"]

In [5]:
X.shape

(530, 40, 3)

In [6]:
model = PatchTSMixerForPrediction.from_pretrained("ibm-granite/granite-timeseries-patchtsmixer")
print(model)



# You can now use the model for your time series tasks
# For example, you can pass input data to the model
# input_data = ...  # Your time series data here
# output = model(input_data)
# print(output)

PatchTSMixerForPrediction(
  (model): PatchTSMixerModel(
    (encoder): PatchTSMixerEncoder(
      (patcher): Linear(in_features=16, out_features=48, bias=True)
      (mlp_mixer_encoder): PatchTSMixerBlock(
        (mixers): ModuleList(
          (0-1): 2 x PatchTSMixerLayer(
            (patch_mixer): PatchMixerBlock(
              (norm): PatchTSMixerNormLayer(
                (norm): LayerNorm((48,), eps=1e-05, elementwise_affine=True)
              )
              (mlp): PatchTSMixerMLP(
                (fc1): Linear(in_features=32, out_features=96, bias=True)
                (dropout1): Dropout(p=0.5, inplace=False)
                (fc2): Linear(in_features=96, out_features=32, bias=True)
                (dropout2): Dropout(p=0.5, inplace=False)
              )
              (gating_block): PatchTSMixerGatedAttention(
                (attn_layer): Linear(in_features=32, out_features=32, bias=True)
                (attn_softmax): Softmax(dim=-1)
              )
            )
      

In [7]:
X[:10, ...].shape

(10, 40, 3)

In [8]:
# Pad the second dimension (sequence length) to 512
seq_len = X.shape[1]
if seq_len < 512:
    pad_width = ((0, 0), (0, 512 - seq_len), (0, 0))
    X_padded = np.pad(X[:10, ...], pad_width, mode='constant')
else:
    X_padded = X[:10, ...]


In [9]:
X_padded = torch.tensor(X_padded, dtype=torch.float32)

In [10]:
X_padded.shape

torch.Size([10, 512, 3])

In [16]:
outputs = model(X_padded)
latent = outputs.last_hidden_state

In [ ]:
num_real_patches = 1 + (X.shape[1] - model.config.patch_length) // model.config.patch_stride


In [33]:
latent = latent.reshape(latent.shape[0], latent.shape[1], -1)